In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.model_selection import train_test_split

In [2]:
# Historical data for delays

hist_data= pd.read_csv('hist_data.csv')

hist_data.head()

,mkt_unique_carrier,weather_delay_carrier_mean,month_carrier_mean,origin-dest,origin-dest-mean,month_carrier
0,AS,1.4,0.01,PAE-LAX,0.01,AS-8
1,AS,1.4,0.01,LAX-PAE,0.01,AS-8
2,AS,1.4,0.01,PAE-PDX,0.02,AS-8
3,AS,1.4,0.01,PDX-PAE,0.01,AS-8
4,AS,1.4,0.01,PAE-PDX,0.02,AS-8


In [3]:
#Reading Unseen data
ftest= pd.read_csv('Flight_test.csv',usecols=[0,1,9,12,14,15])
ftest.head()

,fl_date,mkt_unique_carrier,origin,dest,crs_dep_time,crs_arr_time
0,2020-01-01 00:00:00,WN,ONT,SFO,1810,1945
1,2020-01-01 00:00:00,WN,ONT,SFO,1150,1320
2,2020-01-01 00:00:00,WN,ONT,SJC,2020,2130
3,2020-01-01 00:00:00,WN,ONT,SJC,1340,1455
4,2020-01-01 00:00:00,WN,ONT,SJC,915,1035


In [4]:
# Preparing the X_final file with engineering features:
ftest['month'] = pd.DatetimeIndex(ftest['fl_date']).month
ftest['month_day'] = pd.DatetimeIndex(ftest['fl_date']).day
ftest['week_day'] = pd.DatetimeIndex(ftest['fl_date']).weekday
ftest['year_day'] = pd.DatetimeIndex(ftest['fl_date']).dayofyear
ftest['dep_hour'] = pd.to_datetime(ftest['crs_dep_time'], format='%H%M', errors='coerce').dt.hour
ftest['arr_hour'] = pd.to_datetime(ftest['crs_arr_time'], format='%H%M', errors='coerce').dt.hour

In [5]:
ftest.isnull().sum()


fl_date                  0
mkt_unique_carrier       0
origin                   0
dest                     0
crs_dep_time             0
crs_arr_time             0
month                    0
month_day                0
week_day                 0
year_day                 0
dep_hour               104
arr_hour              1873
dtype: int64

In [6]:
# changing midnight to 0
ftest=ftest.fillna(0)

In [7]:
ftest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660556 entries, 0 to 660555
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   fl_date             660556 non-null  object 
 1   mkt_unique_carrier  660556 non-null  object 
 2   origin              660556 non-null  object 
 3   dest                660556 non-null  object 
 4   crs_dep_time        660556 non-null  int64  
 5   crs_arr_time        660556 non-null  int64  
 6   month               660556 non-null  int32  
 7   month_day           660556 non-null  int32  
 8   week_day            660556 non-null  int32  
 9   year_day            660556 non-null  int32  
 10  dep_hour            660556 non-null  float64
 11  arr_hour            660556 non-null  float64
dtypes: float64(2), int32(4), int64(2), object(4)
memory usage: 50.4+ MB


In [8]:
# adding Historical data

#weather hist.
x=hist_data[['mkt_unique_carrier','weather_delay_carrier_mean']]
x=hist_data.groupby('mkt_unique_carrier')['weather_delay_carrier_mean'].mean()
ftest['weather_delay_carrier_mean']= (ftest['mkt_unique_carrier'].map(x)).round(2)

#Cancellation by carrier & month hist
y=hist_data[['month_carrier','month_carrier_mean']]
y=hist_data.groupby('month_carrier')['month_carrier_mean'].mean()
ftest['month']=ftest['month'].astype(str)
ftest['month_carrier']=ftest['mkt_unique_carrier']+'-'+ftest['month']
ftest['month_carrier_mean']= (ftest['month_carrier'].map(y)).round(2)

# #cancellation Origin-dest hist
z=hist_data[['origin-dest','origin-dest-mean']]
z=hist_data.groupby('origin-dest')['origin-dest-mean'].mean()
ftest['origin-dest']=ftest['origin']+'-'+ftest['dest']
ftest['origin-dest-mean']= (ftest['origin-dest'].map(z)).round(2)



In [9]:
ftest.head()


,fl_date,mkt_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,month,month_day,week_day,year_day,dep_hour,arr_hour,weather_delay_carrier_mean,month_carrier,month_carrier_mean,origin-dest,origin-dest-mean
0,2020-01-01 00:00:00,WN,ONT,SFO,1810,1945,1,1,2,1,18.0,19.0,1.41,WN-1,0.02,ONT-SFO,0.03
1,2020-01-01 00:00:00,WN,ONT,SFO,1150,1320,1,1,2,1,11.0,13.0,1.41,WN-1,0.02,ONT-SFO,0.03
2,2020-01-01 00:00:00,WN,ONT,SJC,2020,2130,1,1,2,1,20.0,21.0,1.41,WN-1,0.02,ONT-SJC,0.01
3,2020-01-01 00:00:00,WN,ONT,SJC,1340,1455,1,1,2,1,13.0,14.0,1.41,WN-1,0.02,ONT-SJC,0.01
4,2020-01-01 00:00:00,WN,ONT,SJC,915,1035,1,1,2,1,9.0,10.0,1.41,WN-1,0.02,ONT-SJC,0.01


In [10]:
#calculating minmum values
z_min=ftest['origin-dest-mean'].min()
y_min=ftest['month_carrier_mean'].min()
x_min=ftest['origin-dest-mean'].min()
print(z_min,y_min,x_min)

0.0 0.0 0.0


In [11]:
ftest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660556 entries, 0 to 660555
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   fl_date                     660556 non-null  object 
 1   mkt_unique_carrier          660556 non-null  object 
 2   origin                      660556 non-null  object 
 3   dest                        660556 non-null  object 
 4   crs_dep_time                660556 non-null  int64  
 5   crs_arr_time                660556 non-null  int64  
 6   month                       660556 non-null  object 
 7   month_day                   660556 non-null  int32  
 8   week_day                    660556 non-null  int32  
 9   year_day                    660556 non-null  int32  
 10  dep_hour                    660556 non-null  float64
 11  arr_hour                    660556 non-null  float64
 12  weather_delay_carrier_mean  660556 non-null  float64
 13  month_carrier 

In [12]:
#Filling NAN with minimums

ftest['month_carrier_mean']=ftest['month_carrier_mean'].fillna(y_min)
ftest['weather_delay_carrier_mean']=ftest['weather_delay_carrier_mean'].fillna(x_min)
ftest['origin-dest-mean']=ftest['origin-dest-mean'].fillna(z_min)

In [13]:
ftest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660556 entries, 0 to 660555
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   fl_date                     660556 non-null  object 
 1   mkt_unique_carrier          660556 non-null  object 
 2   origin                      660556 non-null  object 
 3   dest                        660556 non-null  object 
 4   crs_dep_time                660556 non-null  int64  
 5   crs_arr_time                660556 non-null  int64  
 6   month                       660556 non-null  object 
 7   month_day                   660556 non-null  int32  
 8   week_day                    660556 non-null  int32  
 9   year_day                    660556 non-null  int32  
 10  dep_hour                    660556 non-null  float64
 11  arr_hour                    660556 non-null  float64
 12  weather_delay_carrier_mean  660556 non-null  float64
 13  month_carrier 

In [14]:
X_final=ftest[['fl_date','mkt_unique_carrier','origin','dest','month_day','week_day','dep_hour','weather_delay_carrier_mean','origin-dest-mean','month_carrier_mean']]

In [15]:
X_final.to_csv('Flight_test_processed.csv', index=False)
